In [1]:
import pandas as pd
import numpy as np
import os
import math

In [2]:
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 5.5MB/s 
     |████████████████████████████████| 2.0MB 11.3MB/s 
     |████████████████████████████████| 1.2MB 37.0MB/s 
     |████████████████████████████████| 3.2MB 55.0MB/s 
     |████████████████████████████████| 890kB 51.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=575e2f9eca28f4527d3d3bcd5aa8cb7be058ad3c04aa9ca5cb801d0fc7fe7047
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=de7fffaa90d827844561dceca6f77eb5b40ee2a927bf9641664bde3bc46480df
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [4]:
!pip install faiss-gpu
# !pip install faiss

In [5]:
import faiss
from sentence_transformers import SentenceTransformer, util

In [6]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1') # Load the distil roberta model,whcih was trained on millions of data

In [10]:
!git clone https://github.com/lappanchappan43/Movie-Recommendation.git

Cloning into 'Movie-Recommendation'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 86 (delta 0), reused 5 (delta 0), pack-reused 79
Unpacking objects: 100% (86/86), done.
Checking out files: 100% (36/36), done.


In [11]:
movies = pd.read_csv('Movie-Recommendation/dataset/IMDb movies.csv')
movies.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [12]:
imdb_movies = movies[pd.notnull(movies['description'])]
imdb_movies = imdb_movies.reset_index(drop=True)

In [13]:
imdb_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [14]:
'''
Sentence Transformers will be used to encode sentences into FAISS. To encode into FAISS, we need to have an unique ID (numeric) assigned to each sentence. For this we will use the index of the row of the movie description
'''
imdb_movies['id'] = imdb_movies.index

In [15]:
imdb_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,id
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0,0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0,1
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0,2
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0,3
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0,4


In [16]:
sentences = imdb_movies['description'].tolist()
print(f'Number of sentences: {len(sentences)}')

Number of sentences: 83740


In [17]:
embeddings = model.encode(sentences)
faiss.normalize_L2(embeddings)

In [18]:
print("Shape of the EMbeddings is ",embeddings.shape)

Shape of the EMbeddings is  (83740, 768)


There are many types of FAISS Indices - you can use a Simple FLAT Index, or you can use the concept of Inverted Index and Product Quantisation to index the data.

IVF with Product Quantisation, allows us to efficiently compress the data - it uses nearest neighbour search

With IVF with Product Quantisation, the vectors are first partitioned into clusters using k-means => This step is known as training the index.


In [19]:
dim = 768
ncentroids=50 # This is a hyperparameter, and indicates number of clusters to be split into
m=16 # Hyperparameter
quantizer = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ(quantizer, dim, ncentroids, m, 8)
index.train(embeddings)

In [20]:
print(index.is_trained)
faiss.write_index(index, 'trained.index')

True


In [21]:
# Add embeddings to trained data
ids = imdb_movies['id'].tolist()
ids = np.array(ids)
index.add_with_ids(embeddings, ids)
print(index.ntotal)

83740


In [22]:
faiss.write_index(index, 'block.index')

Let us use the FAISS index to search for similar movie plots


In [27]:
def searchFAISSIndex(data,id_col_name,query,index,nprobe,model,topk=20):
    ## Convert the query into embeddings
    query_embedding=model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)
  
    
    index.nprobe=nprobe
    
    D,I=index.search(query_embedding,topk) 
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [28]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

In [29]:
query="A seventeen-year-old aristocrat falls in love with a kind but poor artist"
search_result=searchFAISSIndex(imdb_movies,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','description','title','cosine_sim','L2_score']]

In [30]:
search_result

,id,description,title,cosine_sim,L2_score
10,30703,A seventeen-year-old aristocrat falls in love ...,Titanic,0.819775,0.600374
5,4689,An English lady falls madly in love with a Fre...,L'avventura viene dal mare,0.798647,0.634591
14,45478,A film director falls in love with the daughte...,Il regista di matrimoni,0.789233,0.649257
0,674,"In Czarist Russia, a peasant officer, resented...",Nella tempesta,0.783309,0.658318
1,682,A young impoverished aristocrat falls in love ...,Sinfonia nuziale,0.782650,0.659317
17,59293,The only daughter of a wealthy businessman fal...,Love U... Mr. Kalakaar!,0.781912,0.660436
6,5393,A beautiful young pianist falls for her mentor.,Gli anni più belli,0.770627,0.677308
9,16748,A young free-spirited drifter falls in love fo...,Blue Jeans,0.766863,0.682842
12,38405,The impoverished brother of a lowly courtesan ...,Mere Mehboob,0.766501,0.683372
7,5643,A good natured soldier with a killer right han...,Soldaten og Jenny,0.766210,0.683799


In [31]:
query="Former Football player  to train an  football team"
search_result=searchFAISSIndex(imdb_movies,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','description','title','cosine_sim','L2_score']]
search_result

,id,description,title,cosine_sim,L2_score
19,83217,A former football player struggles to train a ...,Bigil,0.601647,0.892583
1,14815,Femme fatale martial arts expert teaches the m...,Firecracker,0.528920,0.970649
4,22707,A rookie high school football coach has a hard...,Wildcats,0.520560,0.979225
8,35943,The true story of a newly appointed African-Am...,Il sapore della vittoria - Uniti si vince,0.510388,0.989558
14,54920,"Under the leadership of an ex-footballer, a bu...",Boisko bezdomnych,0.504442,0.995548
0,2790,A college football star struggles to return in...,Saturday's Heroes,0.500292,0.999708
10,43102,An acclaimed college basketball coach is demot...,Un allenatore in palla,0.496874,1.003121
12,44347,A high school swim champion with a troubled pa...,The Guardian,0.495533,1.004457
15,62040,A legendary Egyptian football coach is brought...,The United - Insieme per la vittoria,0.482965,1.016893
11,44342,A school director is forced to train the 'wors...,Dolryeochagi,0.482055,1.017787
